In [1]:
import pandas as pd
import os
import duckdb
from tqdm import tqdm

In [2]:
# Caminho da pasta onde estão os arquivos
pasta = r'C:\Users\Jose Nelio\Documents\GitHub\ipm_2025\data\efd_base\CIPM_EFD-Base-IPM_2027_AnoBase2025_Projetado_20250811'


In [3]:
tabela_completa = []


for arquivo in os.listdir(pasta):
    if arquivo.endswith('.xlsx') and 'Entrada' in arquivo:
        caminho_arquivo = os.path.join(pasta, arquivo)
        print (f'Lendo {arquivo}')
        df = pd.read_excel(caminho_arquivo, sheet_name='Dados EFD Entrada',skiprows=1)
        tabela_completa.append(df)


df_final = pd.concat(tabela_completa, ignore_index=True)
tipos = {
    'Inscrição Estadual': 'string',
    'CPF/CNPJ': 'string',
    'Ano de referência': 'string',
    'Mês de apuração': 'string',
    'Hash do arquivo transmitido': 'string',
    'Origem da Informação': 'string',
    'Cód. Município Origem D100': 'string',
    'CFOP': 'string',
    # 'Valor Operação': 'float',
    # 'Valor BC ICMS': 'float',
    'Declaração utilizado?': 'string',
    'Registro utilizado no calculo VA?': 'string'
}
df_final = df_final.astype(tipos)


df_final.to_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\efd_base\\CIPM_EFD-Base-IPM_2027_AnoBase2025_Projetado_20250811\\efd_base_entradas.parquet')


Lendo EFD-Base-Entrada_IPM_2027_AnoBase2025_Provisório_parte1_20250811.xlsx


c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Lendo EFD-Base-Entrada_IPM_2027_AnoBase2025_Provisório_parte2_20250811.xlsx


c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Lendo EFD-Base-Entrada_IPM_2027_AnoBase2025_Provisório_parte3_20250811.xlsx


...

c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Lendo EFD-Base-Entrada_IPM_2027_AnoBase2025_Provisório_parte5_20250811.xlsx


c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Lendo EFD-Base-Entrada_IPM_2027_AnoBase2025_Provisório_parte6_20250811.xlsx


c:\Users\Jose Nelio\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
df = pd.read_parquet('C:\\Users\\Jose Nelio\\Documents\\GitHub\\ipm_2025\\data\\efd_base\\CIPM_EFD-Base-IPM_2027_AnoBase2025_Projetado_20250811\\efd_base_entradas.parquet')


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998652 entries, 0 to 1998651
Data columns (total 12 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   Inscrição Estadual                 string
 1   CPF/CNPJ                           string
 2   Ano de referência                  string
 3   Mês de apuração                    string
 4   Hash do arquivo transmitido        string
 5   Origem da Informação               string
 6   Cód. Município Origem D100         string
 7   CFOP                               string
 8   Valor Operação                     object
 9   Valor BC ICMS                      object
 10  Declaração utilizado?              string
 11  Registro utilizado no calculo VA?  string
dtypes: object(2), string(10)
memory usage: 183.0+ MB


In [6]:
df

,Inscrição Estadual,CPF/CNPJ,Ano de referência,Mês de apuração,Hash do arquivo transmitido,Origem da Informação,Cód. Município Origem D100,CFOP,Valor Operação,Valor BC ICMS,Declaração utilizado?,Registro utilizado no calculo VA?
0,10003377,31848765000100,2025,1,EC6860DAE3853819C368366A9A0771CA,C190,0.0,1407,"27,32","0,00",SIM,NÃO
1,10003377,31848765000100,2025,1,EC6860DAE3853819C368366A9A0771CA,C190,0.0,1551,"1.785,00","0,00",SIM,NÃO
2,10003377,31848765000100,2025,1,EC6860DAE3853819C368366A9A0771CA,C190,0.0,1556,"117,54","0,00",SIM,NÃO
3,10003377,31848765000100,2025,1,EC6860DAE3853819C368366A9A0771CA,C190,0.0,2102,"1.991,10","0,00",SIM,SIM
4,10003377,31848765000100,2025,1,EC6860DAE3853819C368366A9A0771CA,C190,0.0,2403,"642,30","0,00",SIM,SIM
...,...,...,...,...,...,...,...,...,...,...,...,...
1998647,92044106,32150633000415,2025,7,E593F4A008E4773799010DEC541D325B,C190,0.0,2913,"393.400,00","0,00",SIM,NÃO
1998648,92044106,32150633000415,2025,7,E593F4A008E4773799010DEC541D325B,C190,0.0,2914,"205.000,00","0,00",SIM,NÃO
1998649,92044106,32150633000415,2025,7,E593F4A008E4773799010DEC541D325B,C190,0.0,2915,"735.000,00","0,00",SIM,NÃO
1998650,92044106,32150633000415,2025,7,E593F4A008E4773799010DEC541D325B,C190,0.0,2916,"105.433,38","0,00",SIM,NÃO


In [7]:
import os

def create_gitkeep_files(root_path, folders):
    for folder in folders:
        folder_path = os.path.join(root_path, folder)
        os.makedirs(folder_path, exist_ok=True)  # cria pasta se não existir
        gitkeep_path = os.path.join(folder_path, '.gitkeep')
        with open(gitkeep_path, 'w') as f:
            f.write('')  # cria arquivo .gitkeep vazio

# Caminho raiz do seu repositório local
root_path = r'C:\Users\Jose Nelio\Documents\GitHub\ipm_2025'

# Lista de pastas relativas dentro do repositório
folders = [
    'data/dados_cadastrais',
    'data/efd_base',
    'data/efd_entrega',
    'data/pgdas',
    'data/valor_adicionado'
]

create_gitkeep_files(root_path, folders)
